In [102]:
import pysam
import pandas as pd
from tqdm import tqdm
import pandas as pd
import numpy as np

In [60]:
SF = pysam.Samfile("data/rd/SRR.sam", "rb")

In [67]:
n_skip=0
len_chr={}
with open("data/rd/SRR.sam", 'r') as f:
    for line in f:
        l = line.replace('\n', '').split('\t')
        sn = l[1].replace('SN:', '')
        ln = l[2].replace('LN:', '')
        len_chr[sn] = ln
        if line[:1] == '@':
            n_skip+=1
        else:
            break

In [66]:
sam_list = []
for sam in SF.fetch():
    sam_list.append([SF.getrname(sam.rname), sam.qname, sam.flag, sam.pos, sam.cigar, sam.seq])

In [68]:
SF.close()

In [69]:
df_sam = pd.DataFrame(sam_list, columns=['rname', 'qname', 'flag', 'pos', 'cigar', 'seq'])

In [73]:
df_sam['ln'] = df_sam['rname'].apply(lambda x: int(len_chr[x]) if x in len_chr else np.nan )

In [88]:
df_sam.head()

,rname,qname,flag,pos,cigar,seq,ln
0,chr1,SRR590764.1,147,182976,"[(0, 76)]",TGGGATCTCTGAGCTCAACAAGCCCTCTCTGGGTGGTAGGTGCAGA...,248956422.0
1,chr1,SRR590764.2,83,13026,"[(0, 76)]",CATGAAGGCTGTCAACCAGTCCATAGGCAAGCCTGGCTGCCTCCAG...,248956422.0
2,chr1,SRR590764.2,163,12963,"[(0, 76)]",TTCACTCCCAGCTCAGAGCCCAGGCCAGGGGCCCCCAAGAAAGGCT...,248956422.0
3,chr1,SRR590764.3,99,183736,"[(0, 76)]",AAGGCAGGGCCATCAGGCACCAAAGGGATTCTGCCAGCATAGTGCT...,248956422.0
4,chr1,SRR590764.3,147,183756,"[(0, 76)]",CAAAGGGATTCTGCCAGCATAGTGCTCCTGGACCAGTGATACACCC...,248956422.0


In [94]:
df_sam = df_sam[df_sam['ln'].notna()]

In [105]:
df_sam['pos'] = df_sam['pos'].astype(int)
df_sam['ln'] = df_sam['ln'].astype(int)

In [106]:
max_gene = df_sam['pos'].max() + df_sam['ln'].max()

In [107]:
genom_reads = np.zeros(shape=(max_gene,) )

In [108]:
for i, read in tqdm(df_sam.iterrows(), total=df_sam.shape[0], position=0, leave=True):
    genom_reads[pos:pos+ln] += 1

  0%|                                 | 111/4675631 [00:22<266:13:13,  4.88it/s]


KeyboardInterrupt: 